In [ ]:
import pandas as pd
import statsmodels.api as sm

# 1. 读取处理后的数据
df = pd.read_csv('data_processed.csv')

# 2. 设置因变量（目标变量）
target_col = 'Y染色体浓度'  # 你可以改成其他目标变量

# 3. 选择数值型自变量（排除目标列）
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
if target_col in numeric_cols:
    numeric_cols.remove(target_col)

# 4. 定义 X 和 y
X = df[numeric_cols]
y = df[target_col]

# 5. 添加常数项（截距）
X = sm.add_constant(X)

# 6. 建立 OLS 回归模型
model = sm.OLS(y, X).fit()

# 7. 输出回归结果
print(model.summary())

# 8. 提取显著性变量（p < 0.05）
p_values = model.pvalues
significant_vars = p_values[p_values < 0.05].index.tolist()

# 去掉常数项
if 'const' in significant_vars:
    significant_vars.remove('const')

print("\n显著性变量（p < 0.05）：")
print(significant_vars)

# 9. 保存回归结果到文件
summary_df = pd.DataFrame({
    '变量': model.params.index,
    '系数': model.params.values,
    'p值': model.pvalues.values,
    '显著性': ['显著' if p < 0.05 else '不显著' for p in model.pvalues.values]
})
summary_df.to_csv('regression_results.csv', index=False, encoding='utf-8-sig')

print("\n回归结果已保存到 regression_results.csv")
